In [ ]:
import polars as pl
import networkx as nx
import random
import rtsvg
rt = rtsvg.RACETrack()
from linknode_graph_patterns import LinkNodeGraphPatterns

def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_dfs_, _patterns_, _node_colors_ = [], LinkNodeGraphPatterns(), {}
for _pattern_ in _patterns_.types:
    _g_  = _patterns_.createPattern(_pattern_, prefix=_pattern_+'__')
    for _node_ in _g_.nodes: _node_colors_[_node_] = _pattern_
    _dfs_.append(graphToDataFrame(_g_))
df    = pl.concat(_dfs_)
g     = rt.createNetworkXGraph(df, [('fm','to')])
_pos_ = nx.spring_layout(g) # rtsvg.PolarsForceDirectedLayout(g).results()

In [ ]:
pos_adj, shapes_gen = rt.circlePackGraphComponentPlacement(g, _pos_)
pos_tm              = rt.treeMapGraphComponentPlacement(g, _pos_)
_params_ = {'df':df, 'relationships':[('fm','to')], 'w':512, 'h':512, 'node_size':3, 'node_color':_node_colors_}
rt.tile([rt.link(**_params_, pos=_pos_),rt.link(**_params_, pos=pos_adj, bg_shape_lu=shapes_gen), rt.link(**_params_, pos=pos_tm)])

In [ ]:
#_igl_ = rt.interactiveGraphPanel(df, ln_params={'relationships': [('fm','to')], 'pos':pos_adj})
#_igl_

In [ ]:
import time
_perf_ = {'points':[], 'time':[], 'correct':[]}
for n in range(1,100):
    points = [(random.uniform(-100, 100), random.uniform(-100, 100)) for _ in range(n)]
    t0 = time.time()
    _circle_ = rt.smallestEnclosingCircleApprox(points)
    t1 = time.time()
    _correct_ = True
    for pt in points: 
        if rt.segmentLength((_circle_, pt)) > 1.01 * _circle_[2]: _correct_ = False
    _perf_['points'].append(n), _perf_['time'].append(t1 - t0), _perf_['correct'].append(_correct_)

_svg_ = ['<svg x="0" y="0" width="512" height="512" viewBox="-150 -150 300 300"><rect x="-150" y="-150" width="300" height="300" fill="white" stroke="black" stroke-width="1" />']
for _pt_ in points: _svg_.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="1" fill="black" />')
_svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="None" stroke="red" />')
_svg_.append('</svg>')

rt.tile([rt.xy(pl.DataFrame(_perf_), x_field='points', y_field='time', color_by='correct', dot_size='large', w=512, h=512), ''.join(_svg_)])

In [ ]:
_perf_ = {'points':[], 'time':[]}
for n in [0,1,2,3,4,5,10,100,1000,10000]:
    print(n)
    points = [(random.uniform(-100, 100), random.uniform(-100, 100)) for _ in range(n)]
    t0 = time.time()
    _circle_ = rt.smallestEnclosingCircleApprox(points)
    t1 = time.time()
    if n != 0 and n != 1:
        for pt in points: assert rt.segmentLength((_circle_, pt)) < 1.01 * _circle_[2] # approximation of the circle
    _perf_['points'].append(n), _perf_['time'].append(t1 - t0)
